In [ ]:
import csv
import re

def generate_insert_statements(
    csv_file, 
    create_stream_statement, 
    output_file, 
    table_name):
    # Extract content within parentheses
    stream_content = re.search(r'\((.*?)\)', create_stream_statement, re.DOTALL).group(1)
    
    # Split content into individual column definitions
    columns = re.findall(r'([a-zA-Z_]+)\s+([a-zA-Z_]+)', stream_content)

    # Extract column names and data types, stripping BOM if present
    column_names_create_stream = [col[0].strip() for col in columns]
    data_types_create_stream = {col[0].strip(): col[1] for col in columns}

    # Open the CSV file
    with open(csv_file, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        insert_statements = []

        # Iterate over each row in the CSV
        for row in reader:
            columns = []
            values = []

            # Iterate over each column in the row
            for field, value in row.items():
                if field in column_names_create_stream:
                    # Column name exists in create_stream_statement
                    data_type = data_types_create_stream[field]
                    if value is None:
                        columns.append(field)
                        values.append("")
                    else:
                        if data_type == "DOUBLE" or data_type == "INTEGER":
                            value_str = str(value)
                            if value_str.strip() == "":
                                columns.append(field)
                                values.append("")
                            else:
                                columns.append(field)
                                values.append(value_str)
                        elif data_type == "TIMESTAMP" or data_type == "DATE":
                            columns.append(field)
                            values.append("'" + str(value) + "'")
                        elif data_type == "VARCHAR":
                            columns.append(field)
                            values.append("'" + value.replace("'", "''") + "'")

            # Construct the INSERT INTO statement with the specified table name
            insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"
            insert_statements.append(insert_statement)

    # Write insert statements to output file
    with open(output_file, 'w') as f:
        for statement in insert_statements:
            f.write(statement + '\n')

In [ ]:
create_stream_statement = """
CREATE STREAM financials (
    cust_id VARCHAR KEY,
    monthly_housing_cost INTEGER,
    contact_preference VARCHAR,
    credit_authority_level VARCHAR,
    credit_score INTEGER,
    credit_utilization DOUBLE,
    debt_service_coverage_ratio INTEGER
)
"""

input_data_file = '/Users/briandunn/Desktop/ksqldb projects/Mock Marketing Schema/data/Financials.csv'

output_file_location = '/Users/briandunn/Desktop/ksqldb projects/Mock Marketing Schema/statements/producers/financials_inserts.sql'

generate_insert_statements(
    input_data_file, 
    create_stream_statement, 
    output_file_location,
    "financials"
    )

In [ ]:
create_stream_statement = """
CREATE STREAM household (
    cust_id VARCHAR KEY,
    household_id VARCHAR,
    household_address VARCHAR,
    household_city VARCHAR,
    household_country VARCHAR,
    household_state VARCHAR,
    household_zip_code VARCHAR,
    address_last_changed_date VARCHAR,
    number_of_dependent_adults INTEGER,
    number_of_dependent_children INTEGER,
    family_size INTEGER,
    head_of_household_indicator VARCHAR,
    home_owner_indicator VARCHAR,
    urban_code VARCHAR,
    primary_advisor_id INTEGER
)
"""

input_data_file = '/Users/briandunn/Desktop/ksqldb projects/Mock Marketing Schema/data/Household.csv'

output_file_location = '/Users/briandunn/Desktop/ksqldb projects/Mock Marketing Schema/statements/producers/household_inserts.sql'

generate_insert_statements(
    input_data_file, 
    create_stream_statement, 
    output_file_location,
    "household"
    )